In [46]:
from pathlib import Path
import json


# Paths
data_dir = Path().cwd() / '../data'
bitpanda_dir = data_dir / 'blockfrost/bitpanda'
binance_dir = data_dir / 'blockfrost/binance'

# Load ground truth
with open(data_dir / 'binance_ground_truth.json', 'r') as f:
    binance_ground_truth = json.load(f)

with open(data_dir / 'bitpanda_ground_truth.json', 'r') as f:
    bitpanda_ground_truth = json.load(f)

In [48]:
import requests


# Blockfrost settings
project_id = 'mainnet1nQVUgUPnvLbJPYGJvcUAmKB0T4uurmg'
base_url = 'https://cardano-mainnet.blockfrost.io/api/v0/'
headers = {
    'project_id': project_id
}
# Exchange addresses extracted from withdrawal transactions
exchange_addresses = set()


# Abstract function to handle Blockfrost API response and save data
def handle_blockfrost_api_response(endpoint, data_dir, parameters={}, paging=False):
    page = 1
    while True:
        parameters.update({'page': page})
        response = requests.get(base_url + endpoint, headers=headers, params=parameters)
        if response.status_code == 200:
            data = response.json()
            if not data:  # No more data to fetch
                break
            # Create necessary directories if they don't exist
            path = Path(data_dir / endpoint)
            path.mkdir(parents=True, exist_ok=True)
            # Save data to a JSON file
            with open(path / f'{page}.json', 'w') as f:
                json.dump(data, f, indent=4)
            page += 1
        else:
            print('Error: ', response.status_code)
            break
        if not paging:
            break


# # Fetch and save transactions data for each address
# for address in bitpanda_ground_truth['deposit_addrs']:
#     handle_blockfrost_api_response(f'addresses/{address}/transactions')

# # Fetch and save transaction data for each withdraw transaction
# for tx in bitpanda_ground_truth['withdraw_txs']:
#     handle_blockfrost_api_response(f'txs/{tx}')

In [50]:
# Get address info (stake keys, not necessary can get the stakekeys through cardanoscan)

# address = bitpanda_ground_truth['deposit_addrs'][0]
# handle_blockfrost_api_response(f'addresses/{address}')

In [22]:
# Get addresses associated with stake keys

# stake_address = 'stake1u9n2w8wn6w902s35jvjy6hvs5yevd0a8evc86e48wn2v70cv8j4fn' # Bitpanda wallet stake key 1
# stake_address = 'stake1u9levjdzrnnjxnsaq08lgw0ejndjgkh6rmcvxuej3jkzuwgrrezn8' # Bitpanda wallet stake key 2
# handle_blockfrost_api_response(f'accounts/{stake_address}/addresses', data_dir=bitpanda_dir, paging=True)

In [49]:
# Recursively load all json files in the given directory
for json_file in bitpanda_dir.glob('accounts/**/*.json'):
    with json_file.open('r') as f:
        addresses = json.load(f)
        for item in addresses:
            address = item['address']
            # Fetch transactions from the blockfrost api for this address
            transactions_endpoint = f'addresses/{address}/txs'
            handle_blockfrost_api_response(transactions_endpoint, bitpanda_dir, parameters={}, paging=True)

In [ ]:
import networkx as nx


# Create a new directed graph
G = nx.DiGraph()

# Load transactions data from directory and create graph
for filename in os.listdir('data/'):
    with open('data/' + filename, 'r') as f:
        data = json.load(f)
        # Add nodes and edges to the graph
        for transaction in data:
            G.add_edge(transaction['sender'], transaction['receiver'], weight=transaction['amount'])

# Analyze the graph
print('Number of nodes: ', G.number_of_nodes())
print('Number of edges: ', G.number_of_edges())

# Draw the graph
pos = nx.spring_layout(G)

# Determine node colors
node_colors = []
for node in G.nodes():
    if node in bitpanda_ground_truth['deposit_addrs'] or node in exchange_addresses:
        node_colors.append('blue')  # Exchange addresses
    else:
        node_colors.append('red')  # User addresses

nx.draw_networkx_nodes(G, pos, node_color=node_colors)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos)
plt.show()

# Notes

In [ ]:
binance_ground_truth = {
    'deposit_addrs': [ # Byron address
        'DdzFFzCqrht8FRiABBs8Mpbo2zW2QzEbhQWFXgar9jicwUhmsg8BwFeqSzfztBdVtzhLMppUfJEuzn2NAmmT8ehSntj3ihaf4JL29o7R', # Not traceable, unique address from user
    ],
    'withdraw_txs': [ # Look at inputs
        'a72c1329f2de674bcb262276b097460b69ec02bd9a94f0f9934583e05601d144' # Unique addres, no associated stake keys, transaction contains many unique addresses -> same owner
    ],
    'staking_ids': [ # BNP pools, get info with https://docs.blockfrost.io/#tag/Cardano-Pools/paths/~1pools~1%7Bpool_id%7D/get
        '279d3af9d898261a19860ebc05eed79104915f05ad9f6960b530c3a2', # 45
        'd4970177385f7c3c90f51af3dbc4a865221778b48f23e4dc60dd4a67', # 42
        '625ae8c8aaaead6ceebf262f380245cb61b4a3e96199b27eb9b01ad9', # 35
        '7a990e7ee931f9d548a977d5cf2194f26f7e2f1158752e0bb94a073f', # 34
        '3200ed8ab323773d4d1a64333a0d79b93dc8f3a45c37f88141e34317', # 36
        '72583cf41012446cd1ca839c10049ecd65d31664b11e63d286041d61', # 59
        'c1bef45a2abbb3a862cdd9142236dff38763cc32e91fc8f282823ee6', # 31
        '9418abb33c7cf50778598371bc1f09fe5460d1dedf740faea665f4ac', # 40
        '96a46d5a796a30483492c3717f80b7c6ef87e9525af162ffacf74fdb', # 50
        'ffb3fe58689ae0d70d6fb939a842ee78ecfa10eab561274327e78b7b', # 41
        '639288b5e4bf267ece78e9a4941dd794f50ec54785f84c687b4ab086', # 38
        '21ad7ca84128f182623df17e6cb29ca1e83889e0013f1004cf5beb73', # 43
        'cb9a5414581696cf9176787e99472790355a7de61d67a5643da95a86', # 30
        'e645c034522f32522bf1c64a7f46128c562880abc26abe5f65f25a84', # 69
        '40ff71258d2c73da9e2a5ee3abb9be256e745e7f74c31eb8d4be5299', # 51
        '243499b53228f5953bb906c8af0fb996002dd78712e430aa7d116964', # 32
        'b21ddee9370f41c3b2b71da34f8a57943ba59034a777b80aed503c2b', # 33
        '3caaa49b3f45a914736c393c97c4bb8611ab7d5f66ecb4e214e93606', # 58
        'a7784228ea544bc94455e76f527aaee53266929f0bb2896893a0444f', # 57
        'a250eb51869c6b94cbc507d22e76ed0f35d31ccabdb1725cfbcddb75', # 56
        'b44181287c72d0c7b1bce6a57173cdf739ac77d030bdfc4606138dd2', # 55
        '4b5e02dce5f549a0c332d8b02f91c43a506311fe1cd4498ddcf46bd0', # 21
        '204f49517b4b9df91efb81b030a9b33738f4f0f43819fd2c39cb438b', # 68
        '3d82913d4c8da690aab6af622651f7a659da40af87dcb4b3b98ce830', # 71
        '7e00129d906afbba06a6b279fcf6e4295986a1104b7f5c4046c653bc', # 54
        'caf8502f66536e95138fbf23c14abde1d13090d4aef52b9287f682e8', # 70
        'beb5390bc047831874ad69f615a6f023ea121865c75ec389d6c0a392', # 53
        'e0ac9f1b68d0621a4e6723f369a50e81f27916acb7cd42bc03eaac93', # 73
        '936316c217c6f217640ed8d132cb0ff8133eef73e74e4c28002b9eb3', # 72
        '662796197f741c9b9f0d9830530d01300ee5c63f549784ff2ac81c17', # 52
        '662796197f741c9b9f0d9830530d01300ee5c63f549784ff2ac81c17', # 46
        '82906c841da0f4010b2fa1a2c3726b5571b02eea6abfb48d7c6a4184', # 47
        '2352f8c7d0b9f0bf5b342c135d5b25c910491deec1440bd9d27799a6', # 48
        '2c88e3a7f6279bda114f92dbbcd78b734f0ebff40ff2bdce647e9c53', # 49
        'f99151a5fd003b8bb92ba2af02ed6efec2923b520189cf89e1c264aa', # 60
        'd2b32bb469f6e8bcd6e1e0ba818716d317e3f26d0a93f317b08fdced', # 61
        '11c660b87cb43e4720b053895b3137b4c08f367dbd030127c9c6de9e', # 62
        'c51e4e6cb25065ad0740249c66eafa11e186120602aff1cdca53110a', # 63
        'cf2b0acb72eecb56a05ba6b79922d4b47fedfb93b168a2eb1e5fa260', # 64
        '61d2598400211dcda6878511eb6d84719e5c502c7aa63d500f7a57ce', # 65
        'c1d314a78a43f3d5baa9c2e4105015384211416f220af4a24a8ea3ee', # 66
        '533a710c19bdddc144ab19900d37a81488088259981a5d4885031ce3', # 67
        '6ef88ef4638cc0fd78a9ebd560727f4ef04334c92441cc14c5868a20', # 78
        'e0d44ad70881125fd5e4a99b8f3a38710e7623722d1b9eafab102600', # 77
        '2c08d16625e9db8c13f81032add67387c23c9166fca89e55ef980d64', # 76
        '7d636b630ae3b6d4fb4e46a443bd764384b104790a87db1081dbb15d', # 75
        '704a1d0fe11f573478771b082d6f12043efaffa63319ce3f5cde28c4', # 74
        'cb91386ed557e497bdc45f5c9c7e2dc76172a4f457289195598f9c03', # 1
        '1575d127ca11492f83430cd0e9d204450dc09e587239c52d542cd1fa', # 2
        'd31b91228fc1dedbc3c795d3f7a0201e8eb1af60255e60fdd12acbb4', # 3
        'f7b96ff0848ac1ad86f60fac90e756e54e381cd53ff3e22a20e672f3', # 4
        'c974db0e14dc25c261c2aed2fb034922fe279aa7dcc7c1d108e24161', # 5
        'fb110d21b847e3f4631ef38dbe120b39ab72a0d85ddfe0886478d306', # 7
        '8d5b59b6d7782b1d92c0b954ab3ed718edbf0fe1b48f1acb9ab3a5dd', # 6
        '0ac5fabd6b2f75f5583a4de5b141115ab7d036ee66b13070f6968c81', # 8
        '49cb80cc9bad4696515291ce98a14fe987d7550547ba8fcc1044bea5', # 9
        'ff8ce26a1076026c64292eeb186455b390341b8ea66d7b75928d95f6', # 10
        'f62bc8f972c4bb926df8853694b1b62119ea8ed7c0af38203fca6b09', # 11
        '556e4281187cee08332ad1ed2a0e910b05e0850a2d01a1a9af848d07', # 12
        '576f80a12815b6a28bb271d284825beea6885a1e136d558fc051de17', # 13
        'fa610fe0a56f4f2bca9d08b0b84d3bf6ad2c6a6d263579927d462473', # 14
        '60014da0f8c033b90e9cb27b53f7daf3412877d3ec0e69853ebf1981', # 15
        'd60a72f6f3a1d42a47a292763ca0884f305b56bc880be701049bc46e', # 16
        '39b60916e5f2ccaad362b09e907d9d85050cbd22b94db9943a17bdd9', # 17
        '90264742df54caa49b545dd7971c5f9458048c0617844979eb5f9b10', # 18
        '18532512ac6b647e89dc297deeea305625f8faa1c84a7fe9b7e02f4e', # 19
        '5e964c17799de9ce9a2791cb2434927efa2d66bb418af9f8d6c7fdbe', # 20
        'f85ca74e4a60f1ea259e4d65348916033e337fbb08a93db78e666be3', # 21
        '79bc3877f1fb79bb131cc793e1c3e68b314e9ae2a484e9a13847aedd', # 22
        'fece558da7c7e3ab1e72fdd10cefac1915226e776b42c5dbc9b787e4', # 23
        '518dc473199d020fbfa4f3f9db2012e982e4cb5828487cf09ca82215', # 24
        '943bfed06dc3ef2cbb0ce15bc96047905260fafd680711c6e286a30b', # 25
        '9b991c441a57c186a03e83e4bfd4df6a13394d6f80acdc7ed558a401', # 26
        '9fff445c67c36dfd043330796d634c1a0ada5811584edd1a4d6c281c', # 27
        'de747d8ac5b643793c8027746e16594c98b43dacdfc91d9604d7b191', # 28
        '4d675db60ca11a6e6224aac48e6c0c4f4e637c833cbfe7801ceefafe', # 29
        '79ac7906bb5604b4e8a62a1888ca31351a3b0c4abf3f09660a2bacbc', # 30
        '2ef1a87059981acd82308263348995333b1723e788b5027378c34212', # 31
        'e83506c6c38e272b14b45a577b912f8717788f74afac1d0591a0bce4', # 32
    ]
}

bitpanda_ground_truth = {
    'deposit_addrs': [
        'addr1qxs5n2c76hs3d9wm0uvk9vss8j599ttepl0t58ncaep34ynx5uwa85u274prfyeyf4wepgfjc6l60jes04n2wax5eulsnzlk9d',
        'addr1q863my4xaq3sx2h6fxwp64n4ct4t8ve8pfq7ywnwxzujr4mljey6y888yd8p6q707suln9xmy3d058hscden9r9v9cuswsqelj',
    ],
    'withdraw_txs': [
        '52c2c79b33a71a09b7f38bbd627cc041bd141ab3f9b348e555916ea2537663d7',
        '8444fedd9f29805c3ad542fedce7f76b75a2c67cc58eedae79c04a69672e8de3',
        'bedb077b9380d81ee87b1dfc38a253efc7f08b8c87e2fbe5ff6d0be04e00ce1d',
        'd69c6bd4e56479f2af822c1ce017fc4d06b3ff3e7fc3bdd32908c72e5c88267a',
        '227391ad9a3c958792c599f29e1d07d52e243e4650974b2dcf1a72e3a0bc1334',
        '745bb56473e6ab836d4b8064a485dbc68dc7403533c802e0b4d3c1bea9406312',
        '9df38a441b962bfca15e264d8f23ac902c1da32af500a45f726ce8e855c92fd3',
        '0311c4a1cc557c4970e01ab45f7f6242cd8b53d4d480b9a3ae1976fbcc009b75',
        '650f34567ab15079af586d0edc209071f363a03bde2a8793e602fc2c5ba0bbce',
        'ef217e9c4a67bbb18c5eac5e5b552fe47de4aa811587a4a97bb1e65baa38898f',
        '734490eea2e44c36d4a1f88aed8220b3a17d47af4220b6e530e29f364b6475e0'
    ]
}


import json
from pathlib import Path


data_dir = Path().cwd() / '../data'

with open(data_dir / 'binance_ground_truth.json', 'w') as json_file:
    json.dump(binance_ground_truth, json_file, indent=4)
    
with open(data_dir / 'bitpanda_ground_truth.json', 'w') as json_file:
    json.dump(bitpanda_ground_truth, json_file, indent=4)